In [1]:
# ! pip install pyarrow

# Fetch data

I chose data coming from credit card applications and records. The data is downloaded from Kaggle and can be found at https://www.kaggle.com/rikdifos/credit-card-approval-prediction

## <font>Download files</font>

<font color="red"> You must first install the Kaggle API CLI by running 'pip install kaggle' and also have the Kaggle credentials (kaggle.json) in the same folder of this NB</font>

In [2]:
# set up the Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

!pwd
# 
![ -f data/credit-card-approval-prediction.zip ] || pip install kaggle && cd data && kaggle datasets download rikdifos/credit-card-approval-prediction

!unzip -o data/credit-card-approval-prediction.zip -d data

kaggle.json
/Users/arosero/afroserom/coursera/IBM_advanced_DS
 94%|███████████████████████████████████▋  | 5.00M/5.32M [00:00<00:00, 10.0MB/s]
100%|██████████████████████████████████████| 5.32M/5.32M [00:00<00:00, 8.69MB/s]
Archive:  data/credit-card-approval-prediction.zip
  inflating: data/application_record.csv  
  inflating: data/credit_record.csv  


## Imports

In [3]:
import pandas as pd

## Format conversion

Convert to parquet format for more flexible usage

In [4]:
pd.read_csv("data/application_record.csv").to_parquet("data/application.parquet", index=False)
pd.read_csv("data/credit_record.csv").to_parquet("data/credit_record.parquet", index=False)